### Transforming Canopy Heights
* Separating this for now from the other transformations due to the different methods of data collection for this trait
* Need tests to ensure the correct transformations

### Questions
* What is access level?

In [1]:
import datetime
import pandas as pd
import numpy as np
import sqlite3

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_0 = pd.read_csv('../data/raw/mac_season_4.csv')
# df_0.head()

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (18,26,32,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# for col in df_0.columns.values:
#     print(f'Number of unique variables in {col}: {df_0[col].nunique()}')

In [4]:
# df_0.columns

In [5]:
cols_to_keep = ['result_type', 'id', 'site_id', 'sitename', 'lat', 'lon', 'cultivar_id', 'author', 
                'date', 'time', 'raw_date', 'trait', 'trait_description', 'mean', 'units', 'notes', 
                'access_level', 'cultivar', 'entity', 'method_name']

In [6]:
cols_to_drop = []

for col in df_0.columns.values:
    
    if col not in cols_to_keep:
        cols_to_drop.append(col)

cols_to_drop

['Unnamed: 0',
 'checked',
 'citation_id',
 'treatment_id',
 'city',
 'scientificname',
 'commonname',
 'genus',
 'species_id',
 'citation_year',
 'treatment',
 'month',
 'year',
 'dateloc',
 'n',
 'statname',
 'stat',
 'view_url',
 'edit_url']

In [7]:
df_1 = df_0.drop(labels=cols_to_drop, axis=1)
# df_1.head()

#### Slice dataset where `trait` == `canopy_height`

In [8]:
df_2 = df_1.loc[df_1.trait == 'canopy_height']
# df_2.head()

In [9]:
# df_2.shape

In [10]:
# df_2.entity.unique()

In [12]:
# df_2.notes.unique()

array([nan], dtype=object)

#### Non-null `entity` and `notes` values are not associated with `canopy_height` so can be dropped 

In [14]:
df_3 = df_2.drop(labels=['entity', 'notes'], axis=1)
# df_3.head()

,result_type,id,site_id,sitename,lat,lon,cultivar_id,author,date,time,raw_date,trait,trait_description,mean,units,access_level,cultivar,method_name
51339,traits,6001943979,6000007888,MAC Field Scanner Season 4 Range 30 Column 9 E,33.075590,-111.974913,6000000788,"Newcomb, Maria",2017 Jun 19 (America/Phoenix),[time unspecified or unknown],2017-06-20 00:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",91.0,cm,4,PI329518,Manual canopy height
51340,traits,6001942254,6000007579,MAC Field Scanner Season 4 Range 20 Column 15 W,33.075231,-111.974823,6000000687,"Newcomb, Maria",2017 Jun 4 (America/Phoenix),[time unspecified or unknown],2017-06-05 00:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",99.0,cm,4,PI152828,Manual canopy height
51341,traits,6001940853,6000007579,MAC Field Scanner Season 4 Range 20 Column 15 W,33.075231,-111.974823,6000000687,"Newcomb, Maria",2017 May 29 (America/Phoenix),[time unspecified or unknown],2017-05-30 00:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",63.0,cm,4,PI152828,Manual canopy height
75000,traits,6004768603,6000005801,MAC Field Scanner Season 4 Range 35 Column 14,33.075770,-111.974835,6000000989,"Zongyang, Li",2017 May 20,12:00:00 (America/Phoenix),2017-05-20 14:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",22.0,cm,2,PI619838,3D scanner to 98th quantile height
75001,traits,6004768605,6000005295,MAC Field Scanner Season 4 Range 36 Column 2,33.075806,-111.975032,6000000707,"Zongyang, Li",2017 May 20,12:00:00 (America/Phoenix),2017-05-20 14:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",26.0,cm,2,PI155885,3D scanner to 98th quantile height


#### Separate canopy height measurements
Differences include:
* `method_name`
    * Manual canopy height 
    * 3D scanner to 98th quantile height 
* `author`
    * Newcomb, Maria (manual)
    * Zongyang, Li (scanner)
* `time`
    * unspecified or unknown (manual)
    * 12:00:00 Arizona (scanner)
* `sitename`
    * _sometimes_ E and W specified (manual)
    * no E or W in sitename (scanner)

In [16]:
# could also use != operator in function but only if binary criteria

manual_df = df_3.loc[df_3.method_name == 'Manual canopy height']
scanner_df = df_3.loc[df_3.method_name == '3D scanner to 98th quantile height']

# print(manual_df.shape)
# print(scanner_df.shape)

(4872, 18)
(53184, 18)


In [22]:
print(f'Author for manual df: {manual_df.author.unique()}')
print(f'Author for scanner df: {scanner_df.author.unique()}')

print(' ')

print(f'Unique time values for manual df: {manual_df.time.unique()}')
print(f'Unique time values for scanner df: {scanner_df.time.unique()}')

Author for manual df: ['Newcomb, Maria']
Author for scanner df: ['Zongyang, Li']
 
Time values for manual df: ['[time unspecified or unknown]']
Time values for scanner df: ['12:00:00 (America/Phoenix)']


### For Future Tests

In [18]:
# if slicing df based on certain values, ensure that no rows were overlooked

manual_df.shape[0] + scanner_df.shape[0] == df_3.shape[0]

True